In [1]:
import torch
import torch.nn as nn
import glob
import os
import numpy as np

# ============================================================
# 1. READ DATA
# ============================================================

folder = r"C:\Users\timf\PycharmProjects\aero-surrogate\data\raw\polars"
if os.name == "posix": folder = r"/home/markus/PyCharmMiscProject/aero-surrogate/data/raw/polars"  #Fix für Markus' PC

X_all = []
Y_all = []

files = glob.glob(os.path.join(folder, "*.txt"))
print(f"Found files: {len(files)}")

for file_path in files:
    filename = os.path.basename(file_path)
    parts = filename.replace(".txt", "").split("_")

    try:
        naca = float(parts[0].replace("naca", ""))
        Re = float(parts[1].replace("Re", ""))
        Mach = float(parts[2].replace("M", ""))
    except (IndexError, ValueError) as e:
        print(f"Error parsing {filename}: {e}")
        continue

    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            parts = line.split()
            if len(parts) < 7:
                continue

            try:
                alpha = float(parts[0])
                CL = float(parts[1])
                CD = float(parts[2])
                Top_Xtr = float(parts[5])
                Bot_Xtr = float(parts[6])
            except (ValueError, IndexError):
                continue

            X_all.append([naca, Re, Mach, alpha])
            Y_all.append([CL, CD, Top_Xtr, Bot_Xtr])

X = torch.tensor(X_all, dtype=torch.float32)
Y = torch.tensor(Y_all, dtype=torch.float32)

print(f"✓ Data points read: {X.shape[0]}")
print(f"  X shape: {X.shape}")
print(f"  Y shape: {Y.shape}")

# ============================================================
# 2. NORMALIZATION
# ============================================================

X_mean = X.mean(dim=0)
X_std = X.std(dim=0)
Y_mean = Y.mean(dim=0)
Y_std = Y.std(dim=0)

# Avoid division by zero
X_std = torch.where(X_std == 0, torch.ones_like(X_std), X_std)
Y_std = torch.where(Y_std == 0, torch.ones_like(Y_std), Y_std)

X_norm = (X - X_mean) / X_std
Y_norm = (Y - Y_mean) / Y_std

print(f"\n✓ Normalization completed")
print(f"  X_norm mean: {X_norm.mean(dim=0)}")
print(f"  X_norm std : {X_norm.std(dim=0)}")

# ============================================================
# 3. DEFINE MODELS (4 different architectures)
# ============================================================

# Model 1: Small network (baseline)
class SmallNet(nn.Module):
    """Simple 2-layer network"""
    def __init__(self, input_dim=4, output_dim=4):
        super(SmallNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.network(x)


# Model 2: Medium network
class MediumNet(nn.Module):
    """3-layer network with 64 neurons"""
    def __init__(self, input_dim=4, output_dim=4):
        super(MediumNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.network(x)


# Model 3: Deep network
class DeepNet(nn.Module):
    """4-layer network with 128 neurons"""
    def __init__(self, input_dim=4, output_dim=4):
        super(DeepNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.network(x)


# Model 4: Network with Dropout (regularization)
class DropoutNet(nn.Module):
    """3-layer network with dropout to reduce overfitting"""
    def __init__(self, input_dim=4, output_dim=4, dropout=0.2):
        super(DropoutNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.network(x)


# Instantiate models
models = {
    'small': SmallNet(input_dim=4, output_dim=4),
    'medium': MediumNet(input_dim=4, output_dim=4),
    'deep': DeepNet(input_dim=4, output_dim=4),
    'dropout': DropoutNet(input_dim=4, output_dim=4, dropout=0.2)
}

print(f"\n✓ 4 models created (not trained):")
for name, model in models.items():
    num_params = sum(p.numel() for p in model.parameters())
    print(f"  [{name:8s}] Parameters: {num_params:6d}")

# ============================================================
# 4. SAVE DATA
# ============================================================

save_path = "C:/Users/timf/PycharmProjects/aero-surrogate/data/processed/"
if os.name == "posix": save_path = r"/home/markus/PyCharmMiscProject/aero-surrogate/data/processed/"  #Fix für Markus' PC
os.makedirs(save_path, exist_ok=True)

# Save data as compressed NumPy
np.savez_compressed(
    os.path.join(save_path, 'data_for_training.npz'),
    X=X.numpy(),
    Y=Y.numpy(),
    X_norm=X_norm.numpy(),
    Y_norm=Y_norm.numpy(),
    X_mean=X_mean.numpy(),
    X_std=X_std.numpy(),
    Y_mean=Y_mean.numpy(),
    Y_std=Y_std.numpy()
)

print(f"\n✓ Data saved: {save_path}data_for_training.npz")

# Save model architectures info
model_info = {
    'small': {
        'class': 'SmallNet',
        'input_dim': 4,
        'output_dim': 4,
        'layers': [32],
        'activation': "relu",
        'dropout': 0.0,
        'params': sum(p.numel() for p in models['small'].parameters())
    },
    'medium': {
        'class': 'MediumNet',
        'input_dim': 4,
        'output_dim': 4,
        'layers': [64, 64],
        'activation': "relu",
        'dropout': 0.0,
        'params': sum(p.numel() for p in models['medium'].parameters())
    },
    'deep': {
        'class': 'DeepNet',
        'input_dim': 4,
        'output_dim': 4,
        'layers': [128, 128, 64],
        'activation': "relu",
        'dropout': 0.0,
        'params': sum(p.numel() for p in models['deep'].parameters())
    },
    'dropout': {
        'class': 'DropoutNet',
        'input_dim': 4,
        'output_dim': 4,
        'layers': [64, 64],
        'activation': "relu",
        'dropout': 0.2,
        'params': sum(p.numel() for p in models['dropout'].parameters())
    }
}

np.save(os.path.join(save_path, 'model_info.npy'), model_info)
print(f"✓ Model info saved: {save_path}model_info.npy")

print(f"\nModel comparison (number of parameters):")
for name, info in model_info.items():
    print(f"  {name:8s}: {info['params']:6d} parameters")
print(f"{'='*60}")


Found files: 732
✓ Data points read: 21818
  X shape: torch.Size([21818, 4])
  Y shape: torch.Size([21818, 4])

✓ Normalization completed
  X_norm mean: tensor([-1.4127e-07,  5.4201e-08, -5.8047e-08, -3.9864e-08])
  X_norm std : tensor([1.0000, 1.0000, 1.0000, 1.0000])

✓ 4 models created (not trained):
  [small   ] Parameters:    292
  [medium  ] Parameters:   4740
  [deep    ] Parameters:  25668
  [dropout ] Parameters:   4740

✓ Data saved: /home/markus/PyCharmMiscProject/aero-surrogate/data/processed/data_for_training.npz
✓ Model info saved: /home/markus/PyCharmMiscProject/aero-surrogate/data/processed/model_info.npy

Model comparison (number of parameters):
  small   :    292 parameters
  medium  :   4740 parameters
  deep    :  25668 parameters
  dropout :   4740 parameters
